In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
import imageio

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_mask_classification = load_model("mask_classification.h5")
classes_names = ['MASK', 'NO MASK']
font = cv2.FONT_HERSHEY_SIMPLEX
img_size = (64,64)

In [3]:
def classify_mask(crop):
    """
    classify cropped image : mask or no mask
    """
    crop = cv2.resize(crop,img_size)
    crop = np.expand_dims(crop, axis=0)
    proba = face_mask_classification.predict(tf.convert_to_tensor(crop))
    proba = proba.squeeze()
    class_name = classes_names[0 if proba < 0.5 else 1]
    return proba, class_name

In [4]:
def detect_face_mask(save_gif = False, save_path = None):
    """
    detect face mask using camera
    Params : 
    save_gif : if True save result as gif
    save_path : path to save the gif image
    """
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open camera")
        exit()
    frames = []
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        #reduce camera window
        frame = frame[:280,100:540]
        # Our operations on the frame come here
        # Display the resulting frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, minSize = (100,100))
        for x,y,w,h in faces:
            crop = frame[y:y+h,x:x+w]
            proba, class_name = classify_mask(crop)
            if class_name == "MASK":
                cv2.rectangle(frame, (x+90, y-15), (x, y), (0, 255, 0), -1)
                cv2.putText(frame, f'{class_name}: %.2f '%(1-proba), (x, y-5), font, 0.4, (255, 255, 255), 1)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            else:
                cv2.rectangle(frame, (x+90, y-15), (x, y), (0, 0, 255), -1)
                cv2.putText(frame, f'{class_name}: %.2f '%proba, (x, y-5), font, 0.4, (255, 255, 255), 1)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        if save_gif:
            frames.append(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    if save_gif:
        print("Saving GIF file")
        with imageqio.get_writer(save_path, mode="I") as writer:
            for frame in frames:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                writer.append_data(frame)

In [9]:
detect_face_mask(save_gif = True, save_path = "face_mask_detection.gif")

Saving GIF file
